In [176]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 글꼴깨짐 방지
import matplotlib
import matplotlib.font_manager as fm

import warnings

# 경고 메시지를 무시하고 숨기거나
warnings.filterwarnings(action='ignore')

fm.get_fontconfig_fonts()
font_location = 'C:/Windows/Fonts/malgun.ttf' # For Windows
font_name = fm.FontProperties(fname=font_location).get_name()
matplotlib.rc('font', family=font_name)

In [177]:
df = pd.read_csv('train.csv')
df.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0


In [178]:
def preprocessing(df):
    # 오류 단지코드가 존재하는 행들을  사전에 제거
    df_error =  ['C1095', 'C2051', 'C1218', 'C1894', 'C2483', 'C1502', 'C1988']
    #df_error =  ['C2335', 'C1327']
    df = df[~df['단지코드'].isin(df_error)].reset_index(drop=True)
    df.rename(columns = {'도보 10분거리 내 지하철역 수(환승노선 수 반영)':'지하철','도보 10분거리 내 버스정류장 수':'버스'},inplace=True)
    df.drop(columns=['임대보증금','임대료','자격유형','임대건물구분'],axis = 1,inplace=True)
    지역_비율 = (df.groupby(['지역'])['총세대수'].count())/(df.groupby(['지역'])['총세대수'].count().sum())*100
    지역_비율=지역_비율.reset_index(name='지역_비율')
    공급유형_비율 = (df.groupby(['공급유형'])['총세대수'].count())/(df.groupby(['공급유형'])['총세대수'].count().sum())*100
    공급유형_비율=공급유형_비율.reset_index(name='공급유형_비율')
    df = pd.merge(df,지역_비율, on='지역')
    df = pd.merge(df,공급유형_비율, on='공급유형')
    df.drop(columns=['지역','공급유형','단지코드'],axis = 1,inplace=True)
    df=df.dropna(axis=0)
    df = df[['총세대수', '전용면적', '전용면적별세대수', '공가수', '지하철', '버스', '단지내주차면수', '공급유형_비율',
       '지역_비율', '등록차량수']]
    return df

In [179]:
df = preprocessing(df)
df

,총세대수,전용면적,전용면적별세대수,공가수,지하철,버스,단지내주차면수,공급유형_비율,지역_비율,등록차량수
0,545,33.48,276,17.0,0.0,3.0,624.0,59.944751,12.741713,205.0
1,545,39.60,60,17.0,0.0,3.0,624.0,59.944751,12.741713,205.0
2,545,39.60,20,17.0,0.0,3.0,624.0,59.944751,12.741713,205.0
3,545,46.90,38,17.0,0.0,3.0,624.0,59.944751,12.741713,205.0
4,545,46.90,19,17.0,0.0,3.0,624.0,59.944751,12.741713,205.0
...,...,...,...,...,...,...,...,...,...,...
2891,370,51.99,80,9.0,0.0,3.0,1590.0,0.310773,2.209945,1595.0
2892,370,59.93,142,9.0,0.0,3.0,1590.0,0.310773,2.209945,1595.0
2893,790,51.32,126,13.0,0.0,3.0,673.0,0.310773,2.209945,645.0
2894,790,59.88,49,13.0,0.0,3.0,673.0,0.310773,2.209945,645.0


In [180]:
# Splitting the data into train and test split
from sklearn.model_selection import train_test_split
X = df.iloc[:, 1:-1]
y = df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [183]:
# 데이터 파일 스탠다드 스케일링
from sklearn.preprocessing import StandardScaler
standardScaler = StandardScaler()
print(standardScaler.fit(df))
train_data_standardScaled = standardScaler.transform(df)

StandardScaler()


In [181]:
import lightgbm as lgb

model = lgb.LGBMRegressor()
# model.fit(X_train, y_train)
# pred = model.predict(X_test)

In [182]:
# 스케일링 없이
from time import time
from sklearn.metrics import explained_variance_score,mean_absolute_error,r2_score

start = time()
model.fit(X_train, y_train)
train_time = time() - start
start = time()
y_pred = model.predict(X_test)
predict_time = time()-start    
print(model)
print("\tTraining time: %0.3fs" % train_time)
print("\tPrediction time: %0.3fs" % predict_time)
print("\tExplained variance:", explained_variance_score(y_test, y_pred))
print("\tMean absolute error:", mean_absolute_error(y_test, y_pred))
print("\tR2 score:", r2_score(y_test, y_pred))
print()

LGBMRegressor()
	Training time: 0.079s
	Prediction time: 0.002s
	Explained variance: 0.9749516183438984
	Mean absolute error: 41.81102536265501
	R2 score: 0.9749361079985354



In [184]:
#  스탠다드 스케일링
import lightgbm as lgb

sc_model = lgb.LGBMRegressor()
model.fit(sc_X_train, y_train)
sc_pred = model.predict(sc_X_test)

In [185]:
# 스케일링한 모델 평가 mse 0.8% 잡았음
from time import time
from sklearn.metrics import explained_variance_score,mean_absolute_error,r2_score

start = time()
model.fit(sc_X_train, y_train)
train_time = time() - start
start = time()
y_pred = model.predict(sc_X_test)
predict_time = time()-start    
print(sc_model)
print("\tsc_Training time: %0.3fs" % train_time)
print("\tsc_Prediction time: %0.3fs" % predict_time)
print("\tsc_Explained variance:", explained_variance_score(y_test, y_pred))
print("\tsc_Mean absolute error:", mean_absolute_error(y_test, y_pred))
print("\tsc_R2 score:", r2_score(y_test, y_pred))
print()

LGBMRegressor()
	sc_Training time: 0.078s
	sc_Prediction time: 0.000s
	sc_Explained variance: 0.9756247864193484
	sc_Mean absolute error: 41.057764299546015
	sc_R2 score: 0.9755941987556696



In [186]:
# 오이피클
import joblib

joblib.dump(model, 'lightgbm.pkl')

['lightgbm.pkl']

In [134]:
# 핵심 파라미터들 넣기
params = {'learning_rate': (0.2,0.3,0.4), # 부스팅 각 이터레이션 마다 곱해지는 가중치로 모형 성능과 학습시간에 영향을 준다. 
          'max_depth': [8, 4, 2, 1, -1],# 최대 깊이
          'boosting': ['gbdt'], # ecision tree와 학습 데이터의 일부만 사용하여 부스팅을 수행한다.
          'application':['mean_absolute_error'],
          'min_data_in_leaf':[20], # left가 가지고 있는 최소한의 레코드. 디폴트 값은 20을 해결 할때 사용
          'objective': ['regression'], # 회귀
          'metric': ['mae'],# Mean absolute error
          'maximum_leaves':[31],# 트리의  최대 리프 수. 트리의 최대 깊이, 모형 성능, 학습속도에 영향을 미친다.
          'max_cat_group' :[64], # 카테고리의 수가 클때, 분기 포인트를 찾음
          'is_training_metric': [True],
          'column_sampling':[0.3],# 각 이터레이션에 사용되는 칼럼의 비율
          'num_leaves': [144], # 'maximum_leaves' 와 함께 사용하는 파라미터 
          'feature_fraction': [1,0.7,0.6],# 각 이터레이션에 사용되는 칼럼의 비율. 
          'bagging_fraction': [0.7,0.3,0.1 ],# 일반적으로 훈련 속도를 높이거나, 과적합을 피할때 사용한다.
          'bagging_freq': [5] # 각 이터레이션에 사용되는 행의 비율}

In [135]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(sc_model,params, verbose=10)
sc_model = grid.fit(X,y)
print(sc_model.best_params_,'\n')
print(sc_model.best_estimator_,'\n')

Fitting 5 folds for each of 135 candidates, totalling 675 fits
[CV 1/5; 1/135] START application=mean_absolute_error, bagging_fraction=0.7, bagging_freq=5, boosting=gbdt, column_sampling=0.3, feature_fraction=1, is_training_metric=True, learning_rate=0.2, max_cat_group=64, max_depth=8, maximum_leaves=31, metric=mae, min_data_in_leaf=20, num_leaves=144, objective=regression
[LightGBM] [Warning] Unknown parameter: maximum_leaves
[LightGBM] [Warning] Unknown parameter: max_cat_group
[LightGBM] [Warning] Unknown parameter: column_sampling
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fract

[CV 1/5; 3/135] START application=mean_absolute_error, bagging_fraction=0.7, bagging_freq=5, boosting=gbdt, column_sampling=0.3, feature_fraction=1, is_training_metric=True, learning_rate=0.2, max_cat_group=64, max_depth=2, maximum_leaves=31, metric=mae, min_data_in_leaf=20, num_leaves=144, objective=regression
[LightGBM] [Warning] Unknown parameter: maximum_leaves
[LightGBM] [Warning] Unknown parameter: max_cat_group
[LightGBM] [Warning] Unknown parameter: column_sampling
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=5, subsample_f

[CV 1/5; 5/135] END application=mean_absolute_error, bagging_fraction=0.7, bagging_freq=5, boosting=gbdt, column_sampling=0.3, feature_fraction=1, is_training_metric=True, learning_rate=0.2, max_cat_group=64, max_depth=-1, maximum_leaves=31, metric=mae, min_data_in_leaf=20, num_leaves=144, objective=regression;, score=0.241 total time=   0.0s
[CV 2/5; 5/135] START application=mean_absolute_error, bagging_fraction=0.7, bagging_freq=5, boosting=gbdt, column_sampling=0.3, feature_fraction=1, is_training_metric=True, learning_rate=0.2, max_cat_group=64, max_depth=-1, maximum_leaves=31, metric=mae, min_data_in_leaf=20, num_leaves=144, objective=regression
[LightGBM] [Warning] Unknown parameter: maximum_leaves
[LightGBM] [Warning] Unknown parameter: max_cat_group
[LightGBM] [Warning] Unknown parameter: column_sampling
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0

[CV 3/5; 6/135] START application=mean_absolute_error, bagging_fraction=0.7, bagging_freq=5, boosting=gbdt, column_sampling=0.3, feature_fraction=1, is_training_metric=True, learning_rate=0.3, max_cat_group=64, max_depth=8, maximum_leaves=31, metric=mae, min_data_in_leaf=20, num_leaves=144, objective=regression
[LightGBM] [Warning] Unknown parameter: maximum_leaves
[LightGBM] [Warning] Unknown parameter: max_cat_group
[LightGBM] [Warning] Unknown parameter: column_sampling
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=5, subsample_f

[CV 1/5; 9/135] START application=mean_absolute_error, bagging_fraction=0.7, bagging_freq=5, boosting=gbdt, column_sampling=0.3, feature_fraction=1, is_training_metric=True, learning_rate=0.3, max_cat_group=64, max_depth=1, maximum_leaves=31, metric=mae, min_data_in_leaf=20, num_leaves=144, objective=regression
[LightGBM] [Warning] Unknown parameter: maximum_leaves
[LightGBM] [Warning] Unknown parameter: max_cat_group
[LightGBM] [Warning] Unknown parameter: column_sampling
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=5, subsample_f

...

[CV 4/5; 129/135] START application=mean_absolute_error, bagging_fraction=0.1, bagging_freq=5, boosting=gbdt, column_sampling=0.3, feature_fraction=0.6, is_training_metric=True, learning_rate=0.3, max_cat_group=64, max_depth=1, maximum_leaves=31, metric=mae, min_data_in_leaf=20, num_leaves=144, objective=regression
[LightGBM] [Warning] Unknown parameter: maximum_leaves
[LightGBM] [Warning] Unknown parameter: max_cat_group
[LightGBM] [Warning] Unknown parameter: column_sampling
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] bagging_fraction is set=0.1, subsample=1.0 will be ignored. Current value: bagging_fraction=0.1
[LightGBM] [Warning] bagging_freq is set=5, sub

[LightGBM] [Warning] Unknown parameter: maximum_leaves
[LightGBM] [Warning] Unknown parameter: max_cat_group
[LightGBM] [Warning] Unknown parameter: column_sampling
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] bagging_fraction is set=0.1, subsample=1.0 will be ignored. Current value: bagging_fraction=0.1
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[CV 1/5; 131/135] END application=mean_absolute_error, bagging_fraction=0.1, bagging_freq=5, boosting=gbdt, column_sampling=0.3, feature_fraction=0.6, is_training_metric=True, learning_rate=0.4, max_cat_group=64, max_depth=8, maximum_leaves=31, metric=mae,

[CV 3/5; 132/135] START application=mean_absolute_error, bagging_fraction=0.1, bagging_freq=5, boosting=gbdt, column_sampling=0.3, feature_fraction=0.6, is_training_metric=True, learning_rate=0.4, max_cat_group=64, max_depth=4, maximum_leaves=31, metric=mae, min_data_in_leaf=20, num_leaves=144, objective=regression
[LightGBM] [Warning] Unknown parameter: maximum_leaves
[LightGBM] [Warning] Unknown parameter: max_cat_group
[LightGBM] [Warning] Unknown parameter: column_sampling
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] bagging_fraction is set=0.1, subsample=1.0 will be ignored. Current value: bagging_fraction=0.1
[LightGBM] [Warning] bagging_freq is set=5, sub

[CV 1/5; 134/135] START application=mean_absolute_error, bagging_fraction=0.1, bagging_freq=5, boosting=gbdt, column_sampling=0.3, feature_fraction=0.6, is_training_metric=True, learning_rate=0.4, max_cat_group=64, max_depth=1, maximum_leaves=31, metric=mae, min_data_in_leaf=20, num_leaves=144, objective=regression
[LightGBM] [Warning] Unknown parameter: maximum_leaves
[LightGBM] [Warning] Unknown parameter: max_cat_group
[LightGBM] [Warning] Unknown parameter: column_sampling
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] bagging_fraction is set=0.1, subsample=1.0 will be ignored. Current value: bagging_fraction=0.1
[LightGBM] [Warning] bagging_freq is set=5, sub

[CV 5/5; 135/135] START application=mean_absolute_error, bagging_fraction=0.1, bagging_freq=5, boosting=gbdt, column_sampling=0.3, feature_fraction=0.6, is_training_metric=True, learning_rate=0.4, max_cat_group=64, max_depth=-1, maximum_leaves=31, metric=mae, min_data_in_leaf=20, num_leaves=144, objective=regression
[LightGBM] [Warning] Unknown parameter: maximum_leaves
[LightGBM] [Warning] Unknown parameter: max_cat_group
[LightGBM] [Warning] Unknown parameter: column_sampling
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] bagging_fraction is set=0.1, subsample=1.0 will be ignored. Current value: bagging_fraction=0.1
[LightGBM] [Warning] bagging_freq is set=5, su

In [136]:
sc_params = sc_model.best_params_
sc_params

{'application': 'mean_absolute_error',
 'bagging_fraction': 0.3,
 'bagging_freq': 5,
 'boosting': 'gbdt',
 'column_sampling': 0.3,
 'feature_fraction': 0.6,
 'is_training_metric': True,
 'learning_rate': 0.4,
 'max_cat_group': 64,
 'max_depth': 2,
 'maximum_leaves': 31,
 'metric': 'mae',
 'min_data_in_leaf': 20,
 'num_leaves': 144,
 'objective': 'regression'}

In [137]:
scores_df = pd.DataFrame(grid.cv_results_)
scores_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_application,param_bagging_fraction,param_bagging_freq,param_boosting,param_column_sampling,param_feature_fraction,...,param_objective,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.054921,0.010106,0.002200,4.002095e-04,mean_absolute_error,0.7,5,gbdt,0.3,1,...,regression,"{'application': 'mean_absolute_error', 'baggin...",0.211917,0.873054,0.516804,0.699833,0.704004,0.601122,0.224885,70
1,0.023200,0.000748,0.001600,4.901351e-04,mean_absolute_error,0.7,5,gbdt,0.3,1,...,regression,"{'application': 'mean_absolute_error', 'baggin...",0.080401,0.879309,0.592747,0.718751,0.757846,0.605811,0.278172,59
2,0.018800,0.003430,0.001200,4.002095e-04,mean_absolute_error,0.7,5,gbdt,0.3,1,...,regression,"{'application': 'mean_absolute_error', 'baggin...",0.177704,0.851809,0.656905,0.790171,0.774786,0.650275,0.244564,16
3,0.012600,0.000490,0.001000,1.907349e-07,mean_absolute_error,0.7,5,gbdt,0.3,1,...,regression,"{'application': 'mean_absolute_error', 'baggin...",0.066944,0.709604,0.627678,0.725020,0.677780,0.561405,0.249463,108
4,0.083200,0.007909,0.001600,4.899794e-04,mean_absolute_error,0.7,5,gbdt,0.3,1,...,regression,"{'application': 'mean_absolute_error', 'baggin...",0.240877,0.859906,0.529160,0.690028,0.713443,0.606683,0.210851,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,0.026801,0.001720,0.002000,2.336015e-07,mean_absolute_error,0.1,5,gbdt,0.3,0.6,...,regression,"{'application': 'mean_absolute_error', 'baggin...",0.032923,0.669204,0.351445,0.732586,0.652960,0.487824,0.262911,129
131,0.023200,0.001833,0.001400,4.900571e-04,mean_absolute_error,0.1,5,gbdt,0.3,0.6,...,regression,"{'application': 'mean_absolute_error', 'baggin...",0.234499,0.656775,0.456751,0.746579,0.669192,0.552759,0.185714,115
132,0.018027,0.001532,0.001372,4.586248e-04,mean_absolute_error,0.1,5,gbdt,0.3,0.6,...,regression,"{'application': 'mean_absolute_error', 'baggin...",0.260260,0.707734,0.414591,0.754378,0.615755,0.550543,0.186149,116
133,0.015600,0.001357,0.001400,4.900182e-04,mean_absolute_error,0.1,5,gbdt,0.3,0.6,...,regression,"{'application': 'mean_absolute_error', 'baggin...",0.458160,0.707227,0.467517,0.724511,0.640667,0.599617,0.115173,72


In [138]:
grid.best_score_

0.6850549015830341

In [139]:
import lightgbm as lgb

In [140]:
LGBMRegressor_model = lgb.LGBMRegressor(application= 'mean_absolute_error',
 bagging_fraction= 0.6,
 bagging_freq= 5,
 boosting= 'gbdt',
 feature_fraction= 0.9,
 is_training_metric= True,
 learning_rate= 0.1,
 max_cat_group= 64,
 max_depth= 4,
 metric= 'mae',
 min_data_in_leaf= 20,
 num_leaves= 144,
 objective= 'regression')
LGBMRegressor_model.fit(sc_X_train, y_train)
pred = LGBMRegressor_model.predict(sc_X_test)

In [141]:
from sklearn.metrics import accuracy_score, mean_absolute_error
mean_absolute_error(y_test, pred)

78.31012901565674

In [142]:
# 로버츠 스케일링(아웃라이어 최소화)
from sklearn.preprocessing import RobustScaler
rc = RobustScaler()
rc_X_train = rc.fit_transform(X_train)
rc_X_test = rc.transform(X_test)

In [143]:
import lightgbm as lgb

rc_model = lgb.LGBMRegressor()
model.fit(rc_X_train, y_train)
rc_pred = model.predict(rc_X_test)

In [144]:
# 로버츠스케일링한 모델평가 기준모델보다 mae는 0.7정도,R2 score는 0.006가량 높음
from time import time
from sklearn.metrics import explained_variance_score,mean_absolute_error,r2_score

start = time()
model.fit(rc_X_train, y_train)
train_time = time() - start
start = time()
y_pred = model.predict(rc_X_test)
predict_time = time()-start    
print('rc_model')
print("\tsc_Training time: %0.3fs" % train_time)
print("\tsc_Prediction time: %0.3fs" % predict_time)
print("\tsc_Explained variance:", explained_variance_score(y_test, y_pred))
print("\tsc_Mean absolute error:", mean_absolute_error(y_test, y_pred))
print("\tsc_R2 score:", r2_score(y_test, y_pred))
print()

rc_model
	sc_Training time: 0.130s
	sc_Prediction time: 0.001s
	sc_Explained variance: 0.9755147798730136
	sc_Mean absolute error: 41.1481772945113
	sc_R2 score: 0.9754875971297458



In [145]:
# best params 찾기
from sklearn.model_selection import GridSearchCV

rc_grid = GridSearchCV(rc_model,params, verbose=10)
rc_model = grid.fit(X,y)
print(rc_model.best_params_,'\n')
print(rc_model.best_estimator_,'\n')
print()

Fitting 5 folds for each of 135 candidates, totalling 675 fits
[CV 1/5; 1/135] START application=mean_absolute_error, bagging_fraction=0.7, bagging_freq=5, boosting=gbdt, column_sampling=0.3, feature_fraction=1, is_training_metric=True, learning_rate=0.2, max_cat_group=64, max_depth=8, maximum_leaves=31, metric=mae, min_data_in_leaf=20, num_leaves=144, objective=regression
[LightGBM] [Warning] Unknown parameter: maximum_leaves
[LightGBM] [Warning] Unknown parameter: max_cat_group
[LightGBM] [Warning] Unknown parameter: column_sampling
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fract

[CV 2/5; 3/135] START application=mean_absolute_error, bagging_fraction=0.7, bagging_freq=5, boosting=gbdt, column_sampling=0.3, feature_fraction=1, is_training_metric=True, learning_rate=0.2, max_cat_group=64, max_depth=2, maximum_leaves=31, metric=mae, min_data_in_leaf=20, num_leaves=144, objective=regression
[LightGBM] [Warning] Unknown parameter: maximum_leaves
[LightGBM] [Warning] Unknown parameter: max_cat_group
[LightGBM] [Warning] Unknown parameter: column_sampling
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=5, subsample_f

[CV 1/5; 5/135] END application=mean_absolute_error, bagging_fraction=0.7, bagging_freq=5, boosting=gbdt, column_sampling=0.3, feature_fraction=1, is_training_metric=True, learning_rate=0.2, max_cat_group=64, max_depth=-1, maximum_leaves=31, metric=mae, min_data_in_leaf=20, num_leaves=144, objective=regression;, score=0.241 total time=   0.0s
[CV 2/5; 5/135] START application=mean_absolute_error, bagging_fraction=0.7, bagging_freq=5, boosting=gbdt, column_sampling=0.3, feature_fraction=1, is_training_metric=True, learning_rate=0.2, max_cat_group=64, max_depth=-1, maximum_leaves=31, metric=mae, min_data_in_leaf=20, num_leaves=144, objective=regression
[LightGBM] [Warning] Unknown parameter: maximum_leaves
[LightGBM] [Warning] Unknown parameter: max_cat_group
[LightGBM] [Warning] Unknown parameter: column_sampling
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0

[CV 3/5; 6/135] START application=mean_absolute_error, bagging_fraction=0.7, bagging_freq=5, boosting=gbdt, column_sampling=0.3, feature_fraction=1, is_training_metric=True, learning_rate=0.3, max_cat_group=64, max_depth=8, maximum_leaves=31, metric=mae, min_data_in_leaf=20, num_leaves=144, objective=regression
[LightGBM] [Warning] Unknown parameter: maximum_leaves
[LightGBM] [Warning] Unknown parameter: max_cat_group
[LightGBM] [Warning] Unknown parameter: column_sampling
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=5, subsample_f

[LightGBM] [Warning] Unknown parameter: max_cat_group
[LightGBM] [Warning] Unknown parameter: column_sampling
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[CV 4/5; 8/135] END application=mean_absolute_error, bagging_fraction=0.7, bagging_freq=5, boosting=gbdt, column_sampling=0.3, feature_fraction=1, is_training_metric=True, learning_rate=0.3, max_cat_group=64, max_depth=2, maximum_leaves=31, metric=mae, min_data_in_leaf=20, num_leaves=144, objective=regression;, sc

...

[LightGBM] [Warning] Unknown parameter: maximum_leaves
[LightGBM] [Warning] Unknown parameter: max_cat_group
[LightGBM] [Warning] Unknown parameter: column_sampling
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] bagging_fraction is set=0.1, subsample=1.0 will be ignored. Current value: bagging_fraction=0.1
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[CV 1/5; 129/135] END application=mean_absolute_error, bagging_fraction=0.1, bagging_freq=5, boosting=gbdt, column_sampling=0.3, feature_fraction=0.6, is_training_metric=True, learning_rate=0.3, max_cat_group=64, max_depth=1, maximum_leaves=31, metric=mae,

[CV 4/5; 130/135] START application=mean_absolute_error, bagging_fraction=0.1, bagging_freq=5, boosting=gbdt, column_sampling=0.3, feature_fraction=0.6, is_training_metric=True, learning_rate=0.3, max_cat_group=64, max_depth=-1, maximum_leaves=31, metric=mae, min_data_in_leaf=20, num_leaves=144, objective=regression
[LightGBM] [Warning] Unknown parameter: maximum_leaves
[LightGBM] [Warning] Unknown parameter: max_cat_group
[LightGBM] [Warning] Unknown parameter: column_sampling
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] bagging_fraction is set=0.1, subsample=1.0 will be ignored. Current value: bagging_fraction=0.1
[LightGBM] [Warning] bagging_freq is set=5, su

[CV 5/5; 132/135] START application=mean_absolute_error, bagging_fraction=0.1, bagging_freq=5, boosting=gbdt, column_sampling=0.3, feature_fraction=0.6, is_training_metric=True, learning_rate=0.4, max_cat_group=64, max_depth=4, maximum_leaves=31, metric=mae, min_data_in_leaf=20, num_leaves=144, objective=regression
[LightGBM] [Warning] Unknown parameter: maximum_leaves
[LightGBM] [Warning] Unknown parameter: max_cat_group
[LightGBM] [Warning] Unknown parameter: column_sampling
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] bagging_fraction is set=0.1, subsample=1.0 will be ignored. Current value: bagging_fraction=0.1
[LightGBM] [Warning] bagging_freq is set=5, sub

[CV 3/5; 134/135] END application=mean_absolute_error, bagging_fraction=0.1, bagging_freq=5, boosting=gbdt, column_sampling=0.3, feature_fraction=0.6, is_training_metric=True, learning_rate=0.4, max_cat_group=64, max_depth=1, maximum_leaves=31, metric=mae, min_data_in_leaf=20, num_leaves=144, objective=regression;, score=0.468 total time=   0.0s
[CV 4/5; 134/135] START application=mean_absolute_error, bagging_fraction=0.1, bagging_freq=5, boosting=gbdt, column_sampling=0.3, feature_fraction=0.6, is_training_metric=True, learning_rate=0.4, max_cat_group=64, max_depth=1, maximum_leaves=31, metric=mae, min_data_in_leaf=20, num_leaves=144, objective=regression
[LightGBM] [Warning] Unknown parameter: maximum_leaves
[LightGBM] [Warning] Unknown parameter: max_cat_group
[LightGBM] [Warning] Unknown parameter: column_sampling
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_by

[CV 5/5; 135/135] START application=mean_absolute_error, bagging_fraction=0.1, bagging_freq=5, boosting=gbdt, column_sampling=0.3, feature_fraction=0.6, is_training_metric=True, learning_rate=0.4, max_cat_group=64, max_depth=-1, maximum_leaves=31, metric=mae, min_data_in_leaf=20, num_leaves=144, objective=regression
[LightGBM] [Warning] Unknown parameter: maximum_leaves
[LightGBM] [Warning] Unknown parameter: max_cat_group
[LightGBM] [Warning] Unknown parameter: column_sampling
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] bagging_fraction is set=0.1, subsample=1.0 will be ignored. Current value: bagging_fraction=0.1
[LightGBM] [Warning] bagging_freq is set=5, su

In [146]:
# 똑같이 나옴
rc_params = rc_model.best_params_
rc_params

{'application': 'mean_absolute_error',
 'bagging_fraction': 0.3,
 'bagging_freq': 5,
 'boosting': 'gbdt',
 'column_sampling': 0.3,
 'feature_fraction': 0.6,
 'is_training_metric': True,
 'learning_rate': 0.4,
 'max_cat_group': 64,
 'max_depth': 2,
 'maximum_leaves': 31,
 'metric': 'mae',
 'min_data_in_leaf': 20,
 'num_leaves': 144,
 'objective': 'regression'}

In [ ]:
# 결론 StandardScaler한 기존 모델이 가장 높은 정확도를 지녔다 